#### Input data:  
$$\begin{aligned}
\text{Section information:}
&& I & = \text{{list of course sections}} \\
&& student & = \text{{predicted number of students who will register for each section}} \\
&& PS & = \text{{number of sessions for each section}}  \\
&& TS & = \text{{the timelength of each session in 30-minute block unit (e.g., 3, 6)}}  \\
&& I_2 & = \text{{sections with 2 sessions a week (i.e., MW and TH)}} \\
\\
\text{Professor information:}
&& P & = \text{{list of professors}} \\
&& profSec & = \text{{list of sections taught by each professor}} \\
\\
\text{Classroom information:}
&& J & = \text{{list of classrooms}} \\
&& seat & = \text{{number of seats in each classroom}}  \\
&& B & = \text{{list of buildings}} \\
&& BC & = \text{{list of classrooms by each building}} \\
\\
\text{Day-time information:}
&& D & = \text{{day of week (i.e., M T W H F)}}  \\
&& T & = \text{{indexes representing start time of a section (0 is 8AM and 26 is 9PM}} \\
&& Prime & = \text{{indexes representing start time of prime time section (4 is 10AM and 20 is 6PM}} \\
\end{aligned}$$

#### Decision variables:
Binary variable $X_{ijdt}$ denotes whether we assign course section $i$ to classroom $j$ at day $d$ starting at time $t$.

*Auxiliary variables:*  
Binary variable $Y_{ijdt}$ denotes whether class i is still in session in classroom $j$ at day $d$ at time $t$.

Binary variable $PD_{dp}$ denotes whether professor $p$ has classes on day of week $d$.

Binary variable $PDB_{p,d,b}$ denotes whether professor $p$ has any class in day of week $d$ in building $b$.

Integer variable $PT_{i}$ denotes the number of sessions of section $i$ that starts and ends within prime time. Prime time is defined as 10AM to 6PM.



#### Objective:
Our goal is to balance different components:
- Maximize the average capacity utilization rate
- Minimize the average number of days professors have to teach
- Maximize the prime time classroom utilization

The linear program is thus:

$$\begin{aligned}
\text{maximize} & (\quad 3*\frac{1}{|PS|} \sum_{i\in I}\sum_{j\in J}\sum_{d\in D}\sum_{t\in T}{X_{ijpt}\frac{student_i}{seat_j}} \\
& - \frac{1}{|P|}\sum_{p \in P}\sum_{d \in D}{PD_{p,d}} \\
& + 2* \frac{1}{5*8*60*|J|}\sum_{i\in I}PT_i*TS_i*30) \\
\end{aligned}$$

#### Constraints:
$$\begin{aligned}
\text{1. No start time conflict for classes:}\\
Y_{ijdt} & = \sum_{t' = max(0,t-TS_i+1)}^{t} X_{ijdt'}  && \forall i\in I,j\in J,d\in D,t\in T\\
\sum_{i\in I} y_{ijdt} & \le 1 && \forall j\in J, d\in D, t\in T \\
\\
\text{2. All required sessions must be scheduled:} \\ 
\sum_{j\in J}\sum_{d\in D}\sum_{t\in T}{X_{ijdt}} & = PS_i && \forall i \in I\\
\\
\text{3. Number of students must not exceed total seats:} \\ 
X_{ijdt} * student_i & \le seat_j && \forall i \in I, j \in J, d \in D, t \in T \\
\\
\text{4. No class lasts beyond school time (i.e., 9.30PM):} \\
\sum_{j \in J}\sum_{d \in D}\sum_{t \in \{T_{26}-TS_i+2,...,T_{26}\}} X_{ijdt} & = 0 && \forall i \in I \\
\\
\text{5. Binding clause for 2-session course (MW):} \\ 
X_{ij'M't} & = X_{ij'W't} && \forall i \in I_2, j \in J, t \in T\\
\\
\text{6. Binding clause for 2-session course (TH):} \\
X_{ij'T't} & = X_{ij'H't} && \forall i \in I_2, j \in J, t \in T\\
\\
\text{7. No Friday class for courses with 2 sessions:} \\
\sum_{j\in J}\sum_{t\in T}X_{ij'F't} & = 0 && \forall i \in I_2 \\
\\
\text{8. No teaching conflict for professor:} \\
\sum_{i\in ps}\sum_{j\in J} X_{ijdt} & \le 1 && \forall ps \in profSec, d \in D, t \in T \\
\\
\text{9. Professor to teach in one building in one day of week:} \\
PDB_{pdb} & \ge X_{ijdt} && \forall p \in profSec, i\in p, d \in D, b \in BC, j\in b, t\in T \\
\sum_{b\in B}PDB_{pdb} & \le 1 && \forall p\in P, d\in D \\
\\
\text{10. Number of day professor has to teach (auxiliary):} \\
\sum_{j\in J}\sum_{t\in T}X_{ijdt} & \le PD_{dp} && \forall ps \in profSec, i\in ps, d\in D\\
\\
\text{11. Prime time (auxiliary):} \\
\sum_{j\in J} \sum_{d\in D} \sum_{t\in Prime | t+TS_i\in Prime} \ & \ge PT_i && \forall i\in I\\\
\end{aligned}$$